In [ ]:
import h5py
import numpy as np
import moms_numpy
import nnet_evaluate
import progressbar
import utils
import pylab as plt

In [ ]:
test_data = nnet_evaluate.load_test_data('4B_dayside')

In [ ]:
test_data.keys()

In [ ]:
f1ct = np.ones((32, 16, 32))
counts = np.zeros((32, 16, 32))
weights = {mom: np.zeros((32, 16, 32)) for mom in 'n, vx, vy, vz, txx, tyy, tzz, txy, txz, tyz, nvx, nvy, nvz, ntxx, ntyy, ntzz, ntxz, ntxy, ntyz'.split(', ')}

targets = []

bar = progressbar.ProgressBar()

for i in range(32):
    for j in range(16):
        for k in range(32):
            targets.append((i, j, k))
            
for (i, j, k) in bar(targets):
    counts[:] = 0
    counts[i, j, k] = 1

    result = moms_numpy.idl_moments(
        counts,
        test_data['E'][0],
        test_data['theta'][0],
        test_data['phi'][0],
        extras=True
    )
    
    for mom in result:
        weights[mom][i, j, k] = result[mom]

In [ ]:
print('hi')

In [ ]:
hdf = h5py.File('moms_weights.hdf', 'w')
for mom in weights:
    if mom[0] == 'n':
        hdf[mom] = weights[mom]
hdf.close()

In [ ]:
import moms_fast
import importlib
importlib.reload(moms_fast)

f1ct = utils.get_f1ct({'4B_dayside': test_data}, ['4B_dayside'])
moms_orig, moms_new = [], []
bar = progressbar.ProgressBar()

for i in bar(list(range(test_data['counts'].shape[0]))):
    dist = test_data['counts'][i] * f1ct
    
    if not dist.any():
        continue
    
    result = moms_numpy.idl_moments(
        dist,
        test_data['E'][0],
        test_data['theta'][0],
        test_data['phi'][0],
    )
    moms_orig.append(result)
    
    result =  moms_fast.fast_moments(dist)
    moms_new.append(result)

In [ ]:
from scipy.stats import linregress

for mom in moms_orig[0].keys():
    x = [d[mom] for d in moms_orig]
    y = [d[mom] for d in moms_new]

    r2 = linregress(x, y).rvalue**2
    
    plt.figure()
    plt.title('%s - r2 = %.10f' % (mom, r2))
    plt.plot(x, y, ',')

In [ ]:
for mom in moms_new[0].keys():
    if mom[0] == 'n':
        plt.figure()
        plt.hist( [d[mom] for d in moms_new])
        plt.title(mom)